# Step 1 — Import libs & load file (auto-detect separator)

Why: read the CSV safely (Kaggle marketing files sometimes use ;).


In [16]:
import pandas as pd
import numpy as np

file_path = "marketing_campaign.csv"

# Try to auto-detect separator; fall back to common ones
try:
    df = pd.read_csv(file_path, sep=None, engine='python', encoding='utf-8')
except Exception:
    try:
        df = pd.read_csv(file_path, sep=';', encoding='utf-8')
    except Exception:
        df = pd.read_csv(file_path, sep=',', encoding='utf-8')

print("Loaded shape:", df.shape)

Loaded shape: (2240, 29)


# Step 2 — Quick inspection

Why: see columns, dtypes, missing values and a small preview so you know what you’re working with.

In [17]:
# Columns and dtypes
print("\nColumns:", df.columns.tolist())
print("\nDtypes:\n", df.dtypes)

# Missing values count
print("\nMissing values per column:\n", df.isnull().sum().sort_values(ascending=False).head(20))

# Preview first 8 rows
display(df.head(8))



Columns: ['ID', 'Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome', 'Teenhome', 'Dt_Customer', 'Recency', 'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1', 'AcceptedCmp2', 'Complain', 'Z_CostContact', 'Z_Revenue', 'Response']

Dtypes:
 ID                       int64
Year_Birth               int64
Education               object
Marital_Status          object
Income                 float64
Kidhome                  int64
Teenhome                 int64
Dt_Customer             object
Recency                  int64
MntWines                 int64
MntFruits                int64
MntMeatProducts          int64
MntFishProducts          int64
MntSweetProducts         int64
MntGoldProds             int64
NumDealsPurchases        int64
NumWebPurchases          int64
NumC

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,5,0,0,0,0,0,0,3,11,0
5,7446,1967,Master,Together,62513.0,0,1,09-09-2013,16,520,...,6,0,0,0,0,0,0,3,11,0
6,965,1971,Graduation,Divorced,55635.0,0,1,13-11-2012,34,235,...,6,0,0,0,0,0,0,3,11,0
7,6177,1985,PhD,Married,33454.0,1,0,08-05-2013,32,76,...,8,0,0,0,0,0,0,3,11,0


# Step 3 — Clean column names

Why: consistent column names make future code simpler (lowercase, underscores, no spaces/special chars).

In [18]:
# clean column names
df.columns = [
    str(c).strip().lower().replace(' ', '_').replace('.', '').replace('-', '_')
    for c in df.columns
]
print("Cleaned columns:", df.columns.tolist())


Cleaned columns: ['id', 'year_birth', 'education', 'marital_status', 'income', 'kidhome', 'teenhome', 'dt_customer', 'recency', 'mntwines', 'mntfruits', 'mntmeatproducts', 'mntfishproducts', 'mntsweetproducts', 'mntgoldprods', 'numdealspurchases', 'numwebpurchases', 'numcatalogpurchases', 'numstorepurchases', 'numwebvisitsmonth', 'acceptedcmp3', 'acceptedcmp4', 'acceptedcmp5', 'acceptedcmp1', 'acceptedcmp2', 'complain', 'z_costcontact', 'z_revenue', 'response']


# Step 4 — Convert customer date to datetime

Why: dt_customer is a date string in day-first format; convert to datetime to enable date math.

In [19]:
if 'dt_customer' in df.columns:
    df['dt_customer'] = pd.to_datetime(df['dt_customer'], dayfirst=True, errors='coerce')
    print("dt_customer dtype:", df['dt_customer'].dtype)
    # show any rows where conversion failed
    bad_dates = df['dt_customer'].isna().sum()
    print("dt_customer parse failures (NaT):", bad_dates)
else:
    print("dt_customer column not found.")


dt_customer dtype: datetime64[ns]
dt_customer parse failures (NaT): 0


# Step 5 — Fix obvious categorical inconsistencies

Why: standardize typos / odd values so categories are meaningful (example: 2n Cycle → 2nd Cycle; Alone → Single, Together → Married/Partner, etc.). Adjust mapping to taste.

In [21]:
# Step 5: Standardize categorical/text values

# ---- Clean the 'Education' column ----
if 'Education' in df.columns:
    df['Education'] = (
        df['Education']
        .astype(str)        # ensure string type
        .str.strip()        # remove leading/trailing spaces
        .replace({
            '2n Cycle': '2nd Cycle',
            'Basic': 'Basic Education',
            'Graduation': 'Graduate',
            'Master': 'Postgraduate',
            'PhD': 'Doctorate'
        })
    )

# ---- Clean the 'Marital_Status' column ----
if 'Marital_Status' in df.columns:
    df['Marital_Status'] = (
        df['Marital_Status']
        .astype(str)
        .str.strip()
        .replace({
            'Alone': 'Single',
            'YOLO': 'Single',
            'Absurd': 'Single'
        })
    )

# ---- Verify the changes ----
print("Unique values in 'Education':", df['Education'].unique() if 'Education' in df.columns else "Column not found")
print("Unique values in 'Marital_Status':", df['Marital_Status'].unique() if 'Marital_Status' in df.columns else "Column not found")



Unique values in 'Education': Column not found
Unique values in 'Marital_Status': Column not found


# Step 6 — Examine numeric columns & coerce types

Why: ensure numeric columns are numeric (some may be strings with commas). Update the list below to include any numeric columns in your dataset.

In [22]:
# Candidate numeric column names from this dataset (modify if needed)
numeric_candidates = [
    'year_birth','income','kidhome','teenhome','recency',
    'mntwines','mntfruits','mntmeatproducts','mntfishproducts','mntsweetproducts',
    'mntgoldprods','numdealsaccepted','numwebvisitsmonth','numcatalogpurchases',
    'numstorepurchases','numwebpurchases','numstorevisits','numonlineorders'
]

# Coerce those that exist in df
for col in numeric_candidates:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Show numeric summary
display(df.select_dtypes(include=[np.number]).describe().T)


,count,mean,std,min,25%,50%,75%,max
id,2240.0,5592.159821,3246.662198,0.0,2828.25,5458.5,8427.75,11191.0
year_birth,2240.0,1968.805804,11.984069,1893.0,1959.00,1970.0,1977.00,1996.0
income,2216.0,52247.251354,25173.076661,1730.0,35303.00,51381.5,68522.00,666666.0
kidhome,2240.0,0.444196,0.538398,0.0,0.00,0.0,1.00,2.0
teenhome,2240.0,0.506250,0.544538,0.0,0.00,0.0,1.00,2.0
recency,2240.0,49.109375,28.962453,0.0,24.00,49.0,74.00,99.0
mntwines,2240.0,303.935714,336.597393,0.0,23.75,173.5,504.25,1493.0
mntfruits,2240.0,26.302232,39.773434,0.0,1.00,8.0,33.00,199.0
mntmeatproducts,2240.0,166.950000,225.715373,0.0,16.00,67.0,232.00,1725.0
mntfishproducts,2240.0,37.525446,54.628979,0.0,3.00,12.0,50.00,259.0


# Step 7 — Handle missing income (flag + impute with median)

Why: income had missing values. We add a flag column, then impute the median (robust to outliers). You can change to mean or other method later.

In [23]:
if 'income' in df.columns:
    df['income_missing_flag'] = df['income'].isnull()
    median_income = df['income'].median()
    print("Median income used for imputation:", median_income)
    df['income'] = df['income'].fillna(median_income)
    print("Remaining income NaNs:", df['income'].isnull().sum())
else:
    print("No 'income' column found.")


Median income used for imputation: 51381.5
Remaining income NaNs: 0


# Step 8 — Remove duplicate rows

Why: drop exact duplicates (keeps first occurrence).

In [24]:
before = df.shape[0]
df = df.drop_duplicates(ignore_index=True)
after = df.shape[0]
print(f"Duplicates removed: {before - after}")


Duplicates removed: 0


# Step 9 — Detect outliers in income using IQR and add flag

Why: mark extreme incomes for later handling (capping, transform, or removal).

In [25]:
if 'income' in df.columns:
    Q1 = df['income'].quantile(0.25)
    Q3 = df['income'].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    df['income_outlier'] = (df['income'] < lower) | (df['income'] > upper)
    outlier_count = df['income_outlier'].sum()
    print(f"Income outliers flagged (IQR method): {outlier_count}")
    print(f"IQR lower={lower:.2f}, upper={upper:.2f}")


Income outliers flagged (IQR method): 8
IQR lower=-13587.75, upper=117416.25


# Step 10 — Quick summary & save cleaned CSV

Why: provide a short summary of changes and write the cleaned dataset to disk.

In [27]:
summary = {
    'original_rows': None,  # if you want earlier original shape, store it earlier
    'cleaned_rows': df.shape[0],
    'cleaned_columns': df.shape[1],
    'income_missing_imputed': int(df['income_missing_flag'].sum()) if 'income_missing_flag' in df.columns else 0,
    'income_outliers_flagged': int(df['income_outlier'].sum()) if 'income_outlier' in df.columns else 0
}
print("Cleaning summary:", summary)

clean_path = "marketing_campaign_cleaned.csv"
df.to_csv(clean_path, index=False)
print("Cleaned dataset saved to:", clean_path)


Cleaning summary: {'original_rows': None, 'cleaned_rows': 2240, 'cleaned_columns': 31, 'income_missing_imputed': 24, 'income_outliers_flagged': 8}
Cleaned dataset saved to: marketing_campaign_cleaned.csv
